In [5]:
import pandas as pd
data = pd.read_csv("piracydataset.csv", sep=',')
data.head()

data.drop(columns=['Unnamed: 0'])
#drop discordid and minecraftid 


,age,discordid,minecraftid,client,version
0,14.0,ProGamer300#3331,ProGamer300,Computer ( Java Edition ),Cracked
1,19.0,@shrey#4934,shrey_jain_,Computer ( Java Edition ),Cracked
2,18.0,Skullfaced#8657,Skullfaceddeath,Computer ( Java Edition ),Paid Version
3,17.0,Op DzGhOsT#8719,DzGHoST1737,Computer ( Java Edition ),Cracked
4,15.0,Dwijesh#6020,Dwijesh YT,Mobile ( bedrock version ),Paid Version
...,...,...,...,...,...
1418,17.0,Savager,Asmitraj11,Computer ( Java Edition ),Cracked
1419,15.0,( @KAMALTHESUPERBOY #6357),kamal1112,Computer ( Java Edition ),Cracked
1420,17.0,ATHARVA#9034,SPIDEY_2,Computer ( Java Edition ),Cracked
1421,15.0,GOD_RANVEER#3462,GOD_RANVEER,Computer ( Java Edition ),Cracked
